> **DO YOU USE GITHUB?**  
If True: print('Remember to make your edits in a personal copy of this notebook')  
Else: print('You don't have to understand. Continue your life.')

# Module 8: Web Scraping 3

Now you know how to investigate, download and parse data from a website. Thank you module_6 and module_7. Though, you might still run into websites you are having a difficult time scraping. Getting thorugh such cahllenge using automated browsing will be the main topic of this module. 

We will continue to learn new techniques of parsing unstructured text and HTML. This will help you build ***custom datasets*** within just a few hours or days work, that would have taken ***months*** to curate and clean manually. We will learn how to automate browsing and use regex to parse text without html tags.


Readings for `module 6+7+8`:
- [Python for Data Analysis, chapter 6](https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf)
- [A Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
- [An introduction to web scraping with Python](https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5)
- [Introduction to Web Scraping using Selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

Video materiale from `ISDS 2020`:
- [Web Scraping 1](https://bit.ly/ISDS2021_6)
- [Web Scraping 2](https://bit.ly/ISDS2021_7)
- [Web Scraping 3](https://bit.ly/ISDS2021_8)

Other ressources:
- [Nicklas Webpage](https://nicklasjohansen.netlify.app/)
- [Data Driven Organizational Analysis, Fall 2021](https://efteruddannelse.kurser.ku.dk/course/2021-2022/ASTK18379U)
- [Master of Science (MSc) in Social Data Science](https://www.socialdatascience.dk/education)


## Interactions and Automated Browsing
Sometimes scraping tasks demand interactions (e.g. login, scrolling, clicking), and a no XHR data can be found easily, so you need the browser to execute the scripts before you can get the data. XHR is short for XMLHttpRequest - a JavaScript API - like the one we found in the jobnet.dk exerise.

Here we use the `Selenium` package in combination with the `ChromeDriver` - you can download the latest release [here](https://chromedriver.chromium.org/downloads). It allows you to animate a browser. 

Make sure to download the driver as well as the newest version of Selenium. "pip install selenium" should do the trick. 

Some developers prefer to you [geckodriver](https://github.com/mozilla/geckodriver/releases) as an alternative to `ChromeDriver`.


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


url = 'https:google.com'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [2]:
# You can also download the driver to your computer
# Save it in your working directory and write the code

# import os
# directory = os.getcwd()
# path = os.path.join(directory, 'chromedriver')
# driver = webdriver.Chrome(executable_path=path)

### Benifits from autoamting browsing
1. You can access data that are not directly in the HTML code but that is being generating while browsing
2. You can get thorugh login screens and other scraping barriers
3. You can automate browsing behaviour such as scrolling down

## Example: nboards.dk

In [4]:
# step 1: load the webpage we want to scrape in our virtual browser
url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# step 2: scroll down the page to load more profiles
import time

url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(3)

for i in range(5):
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")





====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.131)


In [6]:
# step 3: save the soup and keep track of runtime

import time
from bs4 import BeautifulSoup

start_time = time.time()

url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

for i in range(5):
    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

soup = BeautifulSoup(driver.page_source, 'lxml')

print("--- %s seconds ---" % round((time.time() - start_time),2))



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


--- 18.61 seconds ---


In [7]:
# step 3: save the soup and keep track of runtime

import time
from bs4 import BeautifulSoup

start_time = time.time()

url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

time.sleep(3)

lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(1)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'lxml')

print("--- %s seconds ---" % round((time.time() - start_time),2))



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


--- 10.9 seconds ---


In [8]:
# step 4: use the soup to generate our mapping of urls (profiles) that we want to scrape

names = soup.find_all('span', {'class': 'name'})

urls = []
for i in range(len(names)):
    temp = 'https://nboard.dk/candidate_profile/'+ str(names[i].text)
    temp = temp.replace(' ','-')
    temp = temp.replace('--','-')
    urls.append(temp)

print(len(urls))
print(urls[1])

20
https://nboard.dk/candidate_profile/Lone-Hass


In [9]:
# step 5: scraping profiles 
import requests
import pandas as pd

start_time = time.time()

name = []
subtitle = []
location = []
resume = []

for i in range(3): #len(urls)
    response = requests.get(urls[i])
    html = response.text
    
    if 'Internal server error' in html:
        continue
    
    soup = BeautifulSoup(html, "html.parser")
    name.append(soup.find('title').text)
    subtitle.append(soup.find('span', {'class': 'sub-title'}).text)
    location.append(soup.find('span', {'class': 'location'}).text)
    resume.append(soup.find('span', {'class': 'resume'}).text)

df = pd.DataFrame({'name':name, 
                   'subtitle':subtitle, 
                   'location':location, 
                   'resume':resume})

print("--- %s seconds ---" % round((time.time() - start_time),2))

df

--- 11.82 seconds ---


,name,subtitle,location,resume
0,Sverrir Sverrisson,Doktor i økonomi med over 30 års arbejdserfaring.,Udlandet,Jeg er Islænding og har min økonomiuddannelse ...
1,Lone Hass,Erfaren topleder og bestyrelsesmedlem,"København, Danmark",Jeg har stor erfaring i styring og udvikling a...


### Next level scrapers

You have know learned some of the fundamentals of collecting and parsing data and should be well suited for your exam project. Though I find it important to adress that you might run into some challenges that we have not learn dealing with yet. Facebook, LinkedIn, Google and all the other big tech firms are battling scrapers and has done all kinds of thing to make it hard for us to steal public data on their sites. I have found som article that you might find interessting.

- [Most Commonly used techniques to Prevent Scraping:](https://medium.com/@betoayesa/using-the-content-as-an-anti-scrape-weapon-draft-9bb10cd30e5c)
- [Advanced Web Scraping Tactics](https://www.pluralsight.com/guides/advanced-web-scraping-tactics-python-playbook)
- [Scraping Sites That Use JavaScript and AJAX](https://oup-arc.com/protected/files/content/file/1505319833942-CH9---Scraping-Sites-that-Use-JavaScript-and-AJAX.pdf)
- [Get Started Scraping LinkedIn With Python and Selenium](https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba)

# Extracting patterns from Raw Text

**Standard string operations**
- `string.split`
- `string.strip`
- `string.replace`

**Regex**
"A regular expression (shortened as regex) is a sequence of characters that define a search pattern. Usually such patterns are used by string-searching algorithms for "find" or "find and replace" operations on strings, or for input validation.

**Examples**
- extract currency and amount from raw text: $ 20, 10.000 dollars 10,000 £
- email addresses: here you want to design a pattern (as above), that captures only the uses of @ within an email.
- urls. Here you are trying to define all the different ways of writing urls (https, http, no http). 
- Dates. Again many variations: 17th of June 2017, 06/17/17 or 17. June 17
- addresses, 
- phone numbers: 8888888 or 88 88 88 88 or +45 88 88 88 88,
- emojiies in text. Capturing all the different ways of expressing smiley faces with one regular expression. 

In [10]:
import re
line = "should we use regex more often? let me know at nj@sodas.ku.dk"
match = re.search(r'[\w\.-]+@[\w\.-]+', line)
match.group(0)

'nj@sodas.ku.dk'

**Ressources**
- Community and interactive playground [here](http://regexr.com/)
- Interactive tutorial [here](https://regexone.com/)
- Lookup all special characters [here](https://www.regular-expressions.info/refquick.html)

**Syntax for build your own expresions**
* \+ = 1 or more times  -- e.g. "a+" will match: "a", and "aaa"
* \* = 0 or more times  -- e.g. "ba*" will match: "b", and "ba", and "baaa"
* {3} = exactly three times --- e.g. "ba{3}" will match "baaa", but not "baa"
* ? = once or none
* \\ = escape character, used to find characters that has special meaning with regex: e.g. \+ \*
* [] = allows you to define a set of characters
* ^ = applied within a set, it becomes the inverse of the set defined. Applied outside a set it entails the beginning of a string. $ entails the end of a string.
* . = any characters except line break
* | = or statement. -- e.g. a|b means find characters a or b.
* \d = digits
* \D = any-non-digits.
* \s = whitespace-separator

Sequences
* (?:) = Defines a Non-capturing group. -- e.g. "(?:abc)+", will match "abc" and "abcabcabc", but not "aabbcc"
* (?=)	= Positive lookahead - only match a certain pattern if a certain pattern comes after it.
* (?!)	= Negative lookahead - only match a certain pattern if **not** a certain pattern comes after it.
* (?<=)	= Positive lookbehind - only match a certain pattern if a certain pattern precedes it.
* (?<!) = Negative lookbehind - only match a certain pattern if **not** a certain pattern precedes it.

In [11]:
import pandas as pd
import re

url = 'https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/danish_review_sample.csv'
df = pd.read_csv(url)

digit_re = re.compile('[0-9]+') # compiled regular expression for matching digits
df['hasNumber'] = df.reviewBody.apply(lambda x: len(digit_re.findall(x))>0) # check if it has a number
sample_string = '\n'.join(df[df.hasNumber].sample(2).reviewBody)
#sample_string

**Comparing expressions**  
Our SODAS collegaue Snorre has developed his own regex library for python. You can either use it by copying his entire class "ExploreRegex" into your notebook or by following this peace of code:
```python 
# download module
url = 'https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py'
response = requests.get(url)
# write script to your folder to create a locate module
with open('explore_regex.py','w') as f:
    f.write(response.text)
# import local module
import ExploreRegex
```

In [12]:
# https://github.com/snorreralund/explore_regex/blob/master/explore_regex.py

import networkx as nx
import re
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
def get_span_overlap(span,span2):
        return min([span[1],span2[1]])-max([span[0],span2[0]])

class ExploreRegex():
    """This module should allow you to compare the differences in matches between regular expressions.
    Default Flags are by design RE.UNICODE and re.DOTALL.
    """
    def __init__(self,sample_string,flags=re.DOTALL|re.UNICODE):
        self.string = sample_string
        self.patterns = []
        self.pattern2span = [] # this container stores all the matches of each pattern.
        self.span_graph = nx.Graph() # defines network that
        self.span2span = nx.Graph()
        self.pattern2pattern = {}
        self.pattern2pattern_soft = {}
        self.pattern2n_match = {}
        self.pattern2chars_matched = {}
        self.pattern2idx = {}
        self.pattern_comparisons = set()
        self.similarity_matrix = []
        self.similarity_matrix_soft = []
        self.flags = flags

    def get_spans(self,pattern):
        "Takes a pattern and locates the spans of the matches."
        if not pattern in self.pattern2chars_matched:
            spans = list(enumerate([result.span() for result in re.finditer(pattern,self.string,flags=self.flags)]))
            print('------ Pattern: %s\t Matched %d patterns -----' %(pattern,len(spans)))
            self.pattern2span.append((pattern,spans))
            match_n = 0
            for num,span in spans:
                match_n+=span[1]-span[0]
            self.pattern2chars_matched[pattern] = match_n
            self.pattern2idx[pattern] = len(self.pattern2chars_matched) -1
            self.pattern2n_match[pattern] = len(spans)
            self.patterns.append(pattern)
        else:
            print('------ Pattern: %s\t Matched %d patterns -----' %(pattern,self.pattern2n_match[pattern]))


    def has_overlap(span,span2):
        "Locates overlap between two pattern spans"
        if span==span2:
            return True
        for val in span:
            if span2[0]<=val<=span2[1]:
                return True
        return False
    def make_overlap_network(self):
        "Constructs Networks between patterns and spans, span2span and pattern2pattern."
        patterns = self.pattern2span
        done = self.pattern_comparisons
        span_g = self.span2span
        pattern2pattern = self.pattern2pattern
        pattern2pattern_soft = self.pattern2pattern_soft
        for i in range(len(patterns)-1):
            pattern,spans = patterns[i]
            for j in range(i+1,len(patterns)):
                if (i,j) in done:
                    continue
                pattern2,spans2 = patterns[j]
                pattern_pair = (i,j)
                for num,span in spans:
                    size = span[1]-span[0]
                    for num2,span2 in spans2:
                        size2 = span2[1]-span2[0]
                        overlap = get_span_overlap(span,span2)
                        if overlap<=0:
                            continue

                        span_sum = size2+size - overlap
                        sim = overlap/span_sum
                        n,n2 = '%d_%d'%(i,num),'%d_%d'%(j,num2)
                        span_g.add_edge(n,n2)
                        span_g[n][n2]['similarity'] = sim
                        span_g.nodes[n]['pattern'] = i
                        span_g.nodes[n2]['pattern'] = j
                        if sim==1:
                            try:
                                pattern2pattern[pattern_pair].add(n)
                                pattern2pattern[pattern_pair].add(n2)
                            except:
                                pattern2pattern[pattern_pair] = set()
                                pattern2pattern[pattern_pair].add(n)
                                pattern2pattern[pattern_pair].add(n2)
                        try:
                            pattern2pattern_soft[pattern_pair].add(n)
                            pattern2pattern_soft[pattern_pair].add(n2)
                        except:
                            pattern2pattern_soft[pattern_pair] = set()
                            pattern2pattern_soft[pattern_pair].add(n)
                            pattern2pattern_soft[pattern_pair].add(n2)

                done.add((i,j))
    def explore_pattern(self,pattern,n_samples=10,context=10,shuffle=True):
        """Prints examples of matches including context. Use the context argument for in- or decreasing the context.
        """
        self.get_spans(pattern)
        idx = self.pattern2idx[pattern]
        spans = [i[1] for i in self.pattern2span[idx][1]]
        n_samples = min([n_samples,len(spans)])
        if shuffle:
            sample = random.sample(spans,n_samples)
        else:
            sample = spans[0:n_samples]
        for start,stop in sample:
            match = self.string[start:stop]
            start,stop = max([start-context,0]),min([stop+context,len(self.string)])
            context_string = self.string[start:stop]
            print('Match: %s\tContext:%s'%(match,context_string))

    def explore_difference(self,pattern,pattern2,method='soft',context = 0):
        """returns two lists of matches only matched by one of the expressions and not in the other.
        Match can be defined as either a perfect match (hard) or overlap between matches (soft).
        Input:
            pattern: regular expression string
            pattern2: regular expression string
            context: defines how much context of the non matches you will see
            method : define the matching method [hard, soft]
        Return:
            list of pattern1 matches not matched by pattern2,list of pattern2 matches not matched by pattern1
        """

        # check if patterns have been matched.
        self.get_spans(pattern)
        self.get_spans(pattern2)
        # add the spans to the overlap network.
        self.make_overlap_network()
        diff = []
        pat_idx,pat_idx2 = self.pattern2idx[pattern],self.pattern2idx[pattern2]
        pattern_pair = tuple(sorted([pat_idx,pat_idx2]))

        if method=='soft':
            if pattern_pair in self.pattern2pattern_soft:
                overlap = self.pattern2pattern_soft[pattern_pair]
            else:
                overlap = set()
        elif method=='hard':
            if pattern_pair in self.pattern2pattern:
                overlap = self.pattern2pattern[pattern_pair]
            else:
                overlap = set()
        else:
            print('Error: you need to define the method as either soft or hard')
            return
        for (num,span) in self.pattern2span[pat_idx][1]:
            n = '%d_%d'%(pat_idx,num)
            if not n in overlap:
                diff.append(self.string[max([span[0]-context,0]):min([span[1]+context,len(self.string)])])
        diff2 = []
        for num,span in self.pattern2span[pat_idx2][1]:
            n = '%d_%d'%(pat_idx2,num)
            if not n in overlap:
                diff2.append(self.string[max([span[0]-context,0]):min([span[1]+context,len(self.string)])])
        print('''Found %d overlaps between the expressions:
        pattern1: %s \t and
        pattern2: %s
        %d included in pattern1 and not in the pattern2
        %d was included in pattern2 and not in pattern1'''%(len(overlap),pattern,pattern2,len(diff),len(diff2)))
        return diff,diff2
    def update_spans(self):
        "Updates matches if a new string is defined."
        self.pattern2span = []
        patterns = list(self.pattern2chars_matched)
        self.pattern2chars_matched = {}
        for pattern in self.patterns:
            self.get_spans(pattern)
    def define_string_sample(self,string):
        "Defines and updates the string to explore matches with."
        self.string = string
        self.update_spans()
    def create_similarity_matrix(self,method='hard'):
        "Creates a directed similarity matrix between patterns defined."
        self.make_overlap_network()
        pat2n = self.pattern2n_match
        patterns = [i[0] for i in self.pattern2span]
        #if len(self.similarity_matrix) == len(patterns): # check if it is already defined.
         #   return None
        if method =='soft':
            g = self.pattern2pattern_soft
            if len(self.similarity_matrix_soft)==len(self.patterns):
                return
        else:
            if len(self.similarity_matrix)==len(self.patterns):
                return
            g = self.pattern2pattern
        mat = np.empty((len(patterns),len(patterns)))
        mat[:] = np.nan
        for i in range(len(patterns)-1):
            n = self.pattern2n_match[patterns[i]]

            for j in range(i+1,len(patterns)):
                n2 = self.pattern2n_match[patterns[j]]
                pattern_pair = (i,j)
                try:
                    overlap = len(g[pattern_pair])/2
                except:
                    overlap = 0
                #sum_ = n+n2 - overlap
                #try:
                #    sim = overlap/sum_
                #except:
                #    sim = np.nan
                if n>0:
                    sim = overlap/n
                else:
                    sim = np.nan
                mat[i][j] = sim
                if n2>0:
                    sim = overlap/n2
                else:
                    sim = np.nan
                mat[j][i] = sim
        if method=='soft':
            self.similarity_matrix_soft = mat
        if method=='hard':
            self.similarity_matrix = mat
    def plot_similarity(self,method='hard'):
        """Plots a directed similarity matrix between patterns.
        The similarity is defined as number of overlapping matches divided by number of matches.
        The definition of overlapping matches between two patterns can be changed from hard (only exact matches) to soft (matches has overlap),
        This will allow you to investigate two different things:
            * Using the 'hard' method you can see how patterns
            * Using 'soft' you can see how expressions narrows the number of accepted patterns.
        method: str ['hard','soft'] parameter for defining overlap between regular expression matches. 'hard' entails exact match, and 'soft' defines match as an overlap between matches.
         """
        patterns = self.patterns
        self.create_similarity_matrix(method)
        if method=='soft':
            mat = self.similarity_matrix_soft
        else:
            mat = self.similarity_matrix
        plt.figure(figsize=(12,8))
        sns.heatmap(mat,cmap='viridis')
        plt.xticks(np.arange(len(patterns))+.5,patterns,rotation=45)
        plt.yticks(np.arange(len(patterns))+.5,patterns,rotation=0)
        plt.title('Similarity Matrix')
    def report(self,method='hard',plot=True):
        """Report the number of matches of each pattern developed and plot a similarity matrix between them.
        The similarity is defined as number of overlapping matches divided by number of matches.
        The definition of overlapping matches between two patterns can be changed from hard (only exact matches) to soft (matches has overlap),
        This will allow you to investigate two different things:
            * Using the 'hard' method you can see how patterns
            * Using 'soft' you can see how expressions narrows the number of accepted patterns.
        method: str ['hard','soft'] parameter for defining overlap between regular expression matches. 'hard' entails exact match, and 'soft' defines match as an overlap between matches.
        """
        for pattern,n in self.pattern2n_match.items():
            print('------ Pattern: %s\t Matched %d patterns -----' %(pattern,n))
        if plot:
            self.plot_similarity(method)
    def compile_pattern(self,pattern):
        """Method to compile the final pattern using the default flags set"""
        return re.compile(pattern,flags=self.flags)

In [13]:
explore_money = ExploreRegex(sample_string)

first = 'kr'
second = '[0-9]+kr'
third = '[0-9]+(?:[,.][0-9]+)?kr'
fourth = '[0-9]+(?:[,.][0-9]+)?\s{0,2}kr'
final = '[0-9]+(?:[,.][0-9]+)?\s{0,5}kr(?:oner)?'

patterns = [first,second,third,fourth,final]

for pattern in patterns:
    explore_money.explore_difference(pattern,patterns[0])

explore_money.explore_pattern(second)

------ Pattern: kr	 Matched 0 patterns -----
------ Pattern: kr	 Matched 0 patterns -----
Found 0 overlaps between the expressions:
        pattern1: kr 	 and
        pattern2: kr
        0 included in pattern1 and not in the pattern2
        0 was included in pattern2 and not in pattern1
------ Pattern: [0-9]+kr	 Matched 0 patterns -----
------ Pattern: kr	 Matched 0 patterns -----
Found 0 overlaps between the expressions:
        pattern1: [0-9]+kr 	 and
        pattern2: kr
        0 included in pattern1 and not in the pattern2
        0 was included in pattern2 and not in pattern1
------ Pattern: [0-9]+(?:[,.][0-9]+)?kr	 Matched 0 patterns -----
------ Pattern: kr	 Matched 0 patterns -----
Found 0 overlaps between the expressions:
        pattern1: [0-9]+(?:[,.][0-9]+)?kr 	 and
        pattern2: kr
        0 included in pattern1 and not in the pattern2
        0 was included in pattern2 and not in pattern1
------ Pattern: [0-9]+(?:[,.][0-9]+)?\s{0,2}kr	 Matched 0 patterns -----
---

# API > Scraping
- Use the API if provided
- Create a developer account
- Learn how to Authenticate by reading their documentation
- Construct your queries and collec the data

**Examples**
- Twitter, YouTube, Reddit, Facebook, Github, Stackexchange, etc.

# Putting something into production?
Right now you are working locally on your computer. That is fine for small project, but some might become data scientist working in organizations that would like to put your work into production. Then we need to be really good at version control (git) but also working with servers and databases. This is obviously not part of this course but you might still want to check [RunDeck ](https://www.rundeck.com/open-source) out. It is a open-source tool you can use to orchestra your scripts. Let's say you want your scraper to run every day at 08:00 am, then you can schedule that job in RunDeck or similar tools.

# Junior Data Scientist Position
A friend of mine, who is a business owner, is looking to hire a junior data scientist. Sent me two or three of the following documents and you might end up getting a kewl job.
1. Code that collects Popular Times data from [this url](https://www.google.com/search?sxsrf=ALeKk02OxWY1MFf29v-44s8Bnozf6EJyHA%3A1597134428470&ei=XFYyX6OTHK_qrgS714u4Ag&q=torvehallerne&oq=torvehallerne&gs_lcp=CgZwc3ktYWIQAzIOCC4QxwEQrwEQywEQkwIyCwguEMcBEK8BEMsBMgsILhDHARCvARDLATILCC4QxwEQrwEQywEyCwguEMcBEK8BEMsBMgsILhDHARCvARDLATIFCAAQywEyCwguEMcBEK8BEMsBMgsILhDHARCvARDLATIFCAAQywE6BwguECcQkwI6BAgAEEM6BwgAEBQQhwI6AggAOg0ILhDHARCvARAnEJMCOgQIIxAnOgUILhCRAjoFCAAQkQI6CAguEMcBEKMCOgoILhDHARCvARBDOgIILjoICC4QxwEQrwE6CwguEMcBEK8BEJMCUJKQAVismgFglZsBaABwAHgBgAGuAYgB7AySAQM0LjmYAQCgAQGqAQdnd3Mtd2l6wAEB&sclient=psy-ab&ved=0ahUKEwijsZGy3ZLrAhUvtYsKHbvrAicQ4dUDCAw&uact=5). Stores and other public places has something called Popular Times when you google them. It indicates how many peoples geolokation are at the place in diffrent time interval. collects data from [this url](https://www.linkedin.com/in/nicklasjohansen/). 
2. Code that collects LinkedIn data from this url together with a simple idea of how you would use LinkedIn data for a social data science project.
3. your_resume.pdf

Reach me by nj@sodas.ku.dk. Deadline is september 1st.


# Master's Thesis Collaboration
I am funded by the [HOPE Project](https://politicalscience.ku.dk/research/projects/hope/) investigating **H**ow Democracies C**ope** with Covid19. We offer thesis collaboration until summer 2023 for student who consider a career in academia. Reach out if you have an idea for your thesis that are within the scope of HOPE.  

Reach me by nj@sodas.ku.dk to start a dialogue.

# Exercise Set 8: Web Scraping 3

In this Exercise Set we shall develop our webscraping skills even further by practicing using `Selenium` while   parsing and navigating html trees using `BeautifoulSoup`. Furthermore we will train extracting information from raw text with no html tags to help, using regex. 

## Exercise Section 8.1: Translating domains into companies
This exercise is about solving a problem that danish companies are facing. They all want to use external data such as customer review data to gain more knowledge about their customers and maybe even use the information as features in their models. There is just one problem: users often create reviews for domains (brand name) and not companies.

> **Ex. 8.1.1:** You work for the danish authorities and are currently staffed to a project where you have to reduce the amount of dangerous toys. You have build a webscraper that collect user reviews form Trustpilot and have identified some websites that got a bad reputation among its users. You belive that the risk of them selling illegal or dangerous toys might be bigger than some of the bg brands with good ratings and decide to investigate them. 

> Go to the website https://www.dk-hostmaster.dk/da/find-domaenenavn with selenium and search for "netbaby.dk". Store the name of the registrant "Euphemia Media" in the variable `company`.

In [21]:
#[Answer 8.1.1]
from selenium.webdriver.common.keys import Keys

url = "https://www.dk-hostmaster.dk/da/find-domaenenavn"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

time.sleep(4)

# Clicking on the cookie popup
cookie_popup = driver.find_element_by_id("CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")
cookie_popup.click()

# Navigating to 'netbaby.dk' through the searchbar
search_box = driver.find_element_by_id('query_domain')
search_box.click()
search_box.send_keys('netbaby.dk')
search_box.send_keys(Keys.RETURN)





====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [22]:
# Extracting name

# company = driver.find_element_by_id("domain_registrant_name")

soup = BeautifulSoup(driver.page_source, 'lxml')


In [23]:
import re

company = str(soup.select('#domain_registrant_name'))
company = re.findall('(?<=>)(.*)(?=<)', company)[0]

company


'Euphemia Media'

> **Ex. 8.1.2:** Now you know who owns the domain and would like to know more about the company `euphemia media`. 

> Go to the Central Business Register website https://datacvr.virk.dk/data/. Figure out how to look up companies by changing the url and then lookup `euphemia media`. Store the CVR number in the variable `cvr` and print it. 

In [24]:
url = "https://datacvr.virk.dk/data/"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

# Navigating to 'Euphemia Media' through the searchbar
search_box = driver.find_element_by_id('edit-search-block-form--2')
search_box.click()
search_box.send_keys(company)
search_box.send_keys(Keys.RETURN)






====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [25]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [26]:
cvr = soup.find_all('div', {'class':'cvr'})[0].text

cvr = cvr.split()[1]

cvr 


'21848875'

> **Ex. 8.1.3:** Congratulations. You are now able to translate domains into companies and by that enrich what ever analysis you want to make. Let's say that you were to build a scraper who could translate thousands of domains. What kind of errors can you imagine running into and how would you mitigate them?

* Some companies might have one name in DK Hostmaster and another in the CVR-database
* Our virtual browser might be shutdown or we would be limited by requesting to frequently

## Exercise Section 8.2: Practicing Regular Expressions.
This exercise is about developing your experience with designing your own regular expressions. This is especially relevant for those of you who are going to work with text data in your exams.

Remember you can always consult the regular expression reference page [here](https://www.regular-expressions.info/refquick.html), if you need to remember or understand a specific symbol. 

We will use a sample of the trustpilot dataset that you practiced collecting in module_7.
You can load it directly into python from the following link: https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv

> **Ex. 8.2.1:** Load the data used in the exercise using the `pd.read_csv` function. (Hint: path to file can be both a url or systempath). 

>Define a variable `sample_string = '\n'.join(df.sample(2000).reviewBody)` as sample of all the reviews that you will practice on.  (Run it once in a while to get a new sample for potential differences).
Imagine we were a company wanting to find the reviews where customers are concerned with the price of a service. They decide to write a regular expression to match all reviews where a currencies and an amount is mentioned. 

> **Ex. 8.2.2:** 
> Write an expression that matches both the dollar-sign (\$) and dollar written literally, and the amount before or after a dollar-sign. Remember that the "$"-sign is a special character in regular expressions. Explore and refine using the explore_pattern function in the package I created called explore_regex. 
```python
import explore_regex as e_re
explore_regex = e_re.Explore_Regex(sample_string) # Initaizlie the Explore regex Class.
explore_regex.explore_pattern(pattern) # Use the .explore_pattern method.
```


Start with exploring the context around digits ("\d") in the data. 

In [32]:
# download module
import requests

url = 'https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py'
response = requests.get(url)

# write script to your folder to create a locate module

with open('explore_regex.py','w') as f:
    f.write(response.text)

# import local module

import explore_regex as e_re 

In [42]:

url = "https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv"
df = pd.read_csv(url)

sample_string = '\n'.join(df.sample(2000).reviewBody)


In [43]:
explore_regex = e_re.ExploreRegex(sample_string) # Initaizlie the Explore regex Class.
pattern = '[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)'
explore_regex.explore_pattern(pattern) # Use the .explore_pattern method.



------ Pattern: [£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)	 Matched 90 patterns -----
Match: $170	Context:ester for $170 less than
Match: $30	Context: The best $30 I could h
Match: £500	Context:aken over £500 out of he
Match: $1	Context:ction for $1 for our o
Match: $2500	Context:asked for $2500 and I was
Match: $2000	Context:price was $2000. The only
Match: 4 dollars	Context:paying $3-4 dollars for a boo
Match: £68	Context: a fee of £68 which inc
Match: $25.00	Context:on of the $25.00 charge fo
Match: $280.78	Context:efund of C$280.78 will be p


> **Ex.8.2.3** Use the .report() method. e_re.report(), and print the all patterns in the development process using the .pattern method - i.e. e_re.patterns 


In [39]:
explore_regex.patterns



['[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)']

> **Ex. 8.2.4** 
Finally write a function that takes in a string and outputs if there is a match. Use the .match function to see if there is a match (hint if does not return a NoneType object - `re.match(pattern,string)!=None`).

> Define a column 'mention_currency' in the dataframe, by applying the above function to the text column of the dataframe. 
*** You should have approximately 310 reviews that matches. - but less is also alright***

> **Ex. 8.2.5** Explore the relation between reviews mentioning prices and the average rating. 

> **Ex. 8.2.6 (extra)** Define a function that outputs the amount mentioned in the review (if more than one the largest), define a new column by applying it to the data, and explore whether reviews mentioning higher prices are worse than others by plotting the amount versus the rating.

In [48]:

currency_re = re.compile(pattern)
def match_currency(string):
    return len(currency_re.findall(string))>0
df['mention_currency'] = df.reviewBody.apply(match_currency)




In [49]:
df.groupby('mention_currency').reviewRating_ratingValue.mean()

mention_currency
False    4.507275
True     2.935275
Name: reviewRating_ratingValue, dtype: float64

> **Ex. 8.2.7:** Now we write a regular expression to extract emoticons from text.
Start by locating all mouths ')' of emoticons, and develop the variations from there. Remember that paranthesis are special characters in regex, so you should use the escape character.

In [50]:
pattern = '[:;][-Oo]?[()D]'
explore_regex.explore_pattern(pattern,context=20)


------ Pattern: [:;][-Oo]?[()D]	 Matched 25 patterns -----
Match: :)	Context:until we receive it :)
Great service to th
Match: :)	Context:nse was a bonus too :)
I am very happy wit
Match: :)	Context:e quick and easy... :) overall is good..
t
Match: :)	Context:
good all keys work :)
the helpfulness of 
Match: :-)	Context:pay any more for it :-)
Perfect ! Merci
I h
Match: :(	Context: black spots on it. :( Love them but, this
Match: :-(	Context:use it for a while. :-(
I requested a plumb
Match: :(	Context:tal experience ever :(
They post honest co
Match: :-)	Context:d and great service :-)
Good place to go fo
Match: :)	Context: that you are doing :)
Thriftbooks is my g
